# ESPIRITO SANTO  

In [1]:
pip install xlsxwriter


In [2]:
import os
import pandas as pd

os.getcwd() # verifica o diretorio

'C:\\Users\\Administrador\\Documents\\PIBIC\\_SCHENIA_2023_2024\\2_dados extraidos\\Causas cid-10'

**Funções**
 

In [3]:
# Ler arquivos no formato que já vem do tabnet ------------------------------------------------------------------------------

def ler_processar_arquivo_csv(caminho_arquivo):
    # Leitura do arquivo CSV com delimitador ";" e primeira linha como cabeçalho
    df = pd.read_csv(caminho_arquivo, delimiter=';', header=0, skiprows=5, encoding='ISO-8859-1')
    
    # Encontra o índice da linha onde está a string 'Total' na coluna 'Causa - CID-BR-10'
    total_index = df[df['Causa - CID-BR-10'] == 'Total'].index[0]

    # Exclui todas as linhas a partir do índice onde está a string 'Total'
    df = df.iloc[:total_index]

    # Muda o formato do dado
    df = pd.concat([df.iloc[:, 0], df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').astype('Int64')], axis=1)

    return df

# Padronizando os arquivos com a mesma quantidade de colunas -----------------------------------------------------------------

# Colunas desejadas
colunas_desejadas = ['0 a 6 dias', '7 a 27 dias', '28 a 364 dias', '1 a 4 anos', '5 a 9 anos', '10 a 14 anos',
                     '15 a 19 anos', '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos', '40 a 44 anos',
                     '45 a 49 anos', '50 a 54 anos', '55 a 59 anos', '60 a 64 anos', '65 a 69 anos', '70 a 74 anos',
                     '75 a 79 anos', '80 anos e mais', 'Total']

# Função para adicionar colunas ausentes com valores zero em um DataFrame
def adicionar_colunas_ausentes(df):
    colunas_a_adicionar = [coluna for coluna in colunas_desejadas if coluna not in df.columns]
    for coluna in colunas_a_adicionar:
        df[coluna] = 0
        
         # Reordena as colunas do DataFrame de acordo com a lista colunas_desejadas
    colunas_ordenadas = ['Causa - CID-BR-10','0 a 6 dias', '7 a 27 dias', '28 a 364 dias', '1 a 4 anos', 
                         '5 a 9 anos', '10 a 14 anos','15 a 19 anos', '20 a 24 anos', '25 a 29 anos',
                         '30 a 34 anos', '35 a 39 anos', '40 a 44 anos','45 a 49 anos', '50 a 54 anos',
                         '55 a 59 anos', '60 a 64 anos', '65 a 69 anos', '70 a 74 anos','75 a 79 anos',
                         '80 anos e mais', 'Total']       
    df = df.reindex(columns=colunas_ordenadas)
    
    return df


# Juntandos os três df com base no Causa - CID-BR-10, somando e depois divido por três---------------------------------------------

import pandas as pd
import math

def processar_dataframes(df_1, df_2, df_3):
    # Realizar a fusão dos primeiros dois DataFrames com base na coluna 'Grupo CID-10'
    df_merged_12 = pd.merge(df_1, df_2, left_on='Causa - CID-BR-10', right_on='Causa - CID-BR-10', how='outer')
    
    # Realizar a fusão dos três DataFrames com base na coluna 'Grupo CID-10'
    df_merged_123 = pd.merge(df_merged_12, df_3, left_on='Causa - CID-BR-10', right_on='Causa - CID-BR-10', how='outer')

    # Obter a lista de todas as colunas
    colunas = df_merged_123.columns

    # Dicionário para armazenar os resultados somados
    resultados_soma = {}

    # Iterar sobre as colunas
    for col in colunas:
        # Separar o prefixo (parte antes do "_") da coluna
        prefixo = col.split('_')[0]

        # Verificar se o prefixo já está no dicionário
        if prefixo in resultados_soma:
            # Se sim, somar a coluna atual
            resultados_soma[prefixo] += df_merged_123[col].fillna(0)
        else:
            # Se não, adicionar a coluna ao dicionário
            resultados_soma[prefixo] = df_merged_123[col].fillna(0)

    # Criar um novo DataFrame com os resultados somados
    df_resultado_soma = pd.DataFrame(resultados_soma)

    # Dividir a soma por três
    # Atribuir o resultado da divisão às colunas da segunda até a 22
    df_resultado_soma[df_resultado_soma.columns[1:22]] = df_resultado_soma[df_resultado_soma.columns[1:22]] / 3
    #df_resultado_soma[df_resultado_soma.columns[1:22]] = df_resultado_soma[df_resultado_soma.columns[1:22]].apply(lambda x: x.apply(math.ceil).astype(int))

    
    #trata os . antes do nome da doença 
    df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')
    # ordena as doenças por ordem alfabeticas
    df_resultado_soma = df_resultado_soma.sort_values(by='Causa - CID-BR-10')
    
    # Ordenando o DataFrame de forma correta para corrigir os dados 
    # Aplicando regex para extrair o número da primeira coluna
    df_resultado_soma['numero'] = df_resultado_soma['Causa - CID-BR-10'].str.extract(r'(\d+)')
    # Aplicando regex para extrair o intervalo da primeira coluna
    df_resultado_soma['intervalo'] = df_resultado_soma['Causa - CID-BR-10'].str.extract(r'(\d+-\d+)')
    df_resultado_soma['numero_final'] = df_resultado_soma['intervalo'].str.extract(r'-(\d+)')

    df_resultado_soma = df_resultado_soma.sort_values(by=['numero', 'numero_final'], ascending=[True, False])
    df_resultado_soma.drop(columns=['numero','intervalo','numero_final'], inplace=True)
    
    # adicionando a coluna <1
    df_resultado_soma['<1'] = df_resultado_soma['0 a 6 dias'] + df_resultado_soma['7 a 27 dias'] + df_resultado_soma['28 a 364 dias']
    df_resultado_soma.drop(columns=['0 a 6 dias', '7 a 27 dias', '28 a 364 dias'], inplace=True)
    
    #reordenando o df
         # Reordena as colunas do DataFrame de acordo com a lista colunas_desejadas
    colunas_ordenadas = ['Causa - CID-BR-10','<1', '1 a 4 anos', 
                         '5 a 9 anos', '10 a 14 anos','15 a 19 anos', '20 a 24 anos', '25 a 29 anos',
                         '30 a 34 anos', '35 a 39 anos', '40 a 44 anos','45 a 49 anos', '50 a 54 anos',
                         '55 a 59 anos', '60 a 64 anos', '65 a 69 anos', '70 a 74 anos','75 a 79 anos',
                         '80 anos e mais', 'Total']       
    df_resultado_soma = df_resultado_soma.reindex(columns=colunas_ordenadas)
    
    #Estraindo apenas as linha com todas as letras em maiusculo
    #letra = df_resultado_soma['Causa - CID-BR-10'].str.isupper()
    #df_final = df_resultado_soma[letra]
    
    return df_resultado_soma


-------

# Intermediario Adjacente 

## Feminino

* **Resposta para 2018**

In [4]:
# lendo arquivos 
caminho_arquivo_1 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_F_2017.csv'
caminho_arquivo_2 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_F_2018.csv'
caminho_arquivo_3 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_F_2019.csv'

df_1 = ler_processar_arquivo_csv(caminho_arquivo_1)
df_2 = ler_processar_arquivo_csv(caminho_arquivo_2)
df_3 = ler_processar_arquivo_csv(caminho_arquivo_3)

In [5]:
# Adicionar colunas padrão aos DataFrames
df_1 = adicionar_colunas_ausentes(df_1)
df_2 = adicionar_colunas_ausentes(df_2)
df_3 = adicionar_colunas_ausentes(df_3)

In [6]:
# Juntando os df e divdindo por 3 
df_resultado = processar_dataframes(df_1, df_2, df_3)
IA_F_2018 = df_resultado

C:\Users\Administrador\AppData\Local\Temp\ipykernel_12768\4064576934.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
  df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')


In [7]:
IA_F_2018.head(1)

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.0,0.0,0.333333,0.0,0.0,0.0,0.333333,0.0,0.0,0.666667,0.333333,0.333333,0.333333,0.0,0.0,0.333333,0.333333,1.333333,4.333333


* **Resposta para 2021**

In [8]:
# lendo arquivos 
caminho_arquivo_1 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_F_2020.csv'
caminho_arquivo_2 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_F_2021.csv'
caminho_arquivo_3 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_F_2022.csv'

df_1 = ler_processar_arquivo_csv(caminho_arquivo_1)
df_2 = ler_processar_arquivo_csv(caminho_arquivo_2)
df_3 = ler_processar_arquivo_csv(caminho_arquivo_3)

In [9]:
# Adicionar colunas padrão aos DataFrames
df_1 = adicionar_colunas_ausentes(df_1)
df_2 = adicionar_colunas_ausentes(df_2)
df_3 = adicionar_colunas_ausentes(df_3)

In [10]:
# Juntando os df e divdindo por 3 
df_resultado = processar_dataframes(df_1, df_2, df_3)
IA_F_2021 = df_resultado

C:\Users\Administrador\AppData\Local\Temp\ipykernel_12768\4064576934.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
  df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')


In [11]:
IA_F_2021.head(1)

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.0,0.0,0.0,0.0,0.333333,0.0,0.333333,0.666667,1.333333,1.333333,2.0,2.666667,2.666667,2.333333,6.333333,3.333333,5.333333,7.0,35.666667


## Masculino

* **Resposta para 2018**

In [12]:
# lendo arquivos 
caminho_arquivo_1 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_M_2017.csv'
caminho_arquivo_2 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_M_2018.csv'
caminho_arquivo_3 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_M_2019.csv'

df_1 = ler_processar_arquivo_csv(caminho_arquivo_1)
df_2 = ler_processar_arquivo_csv(caminho_arquivo_2)
df_3 = ler_processar_arquivo_csv(caminho_arquivo_3)

In [13]:
# Adicionar colunas padrão aos DataFrames
df_1 = adicionar_colunas_ausentes(df_1)
df_2 = adicionar_colunas_ausentes(df_2)
df_3 = adicionar_colunas_ausentes(df_3)

In [14]:
# Juntando os df e divdindo por 3 
df_resultado = processar_dataframes(df_1, df_2, df_3)
IA_M_2018 = df_resultado

C:\Users\Administrador\AppData\Local\Temp\ipykernel_12768\4064576934.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
  df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')


In [15]:
IA_M_2018.head(1)

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.333333,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,0.333333,0.0,1.0,0.666667,0.333333,0.333333,0.333333,0.666667,0.333333,5.666667


* **Resposta para 2021**

In [16]:
# lendo arquivos 
caminho_arquivo_1 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_M_2020.csv'
caminho_arquivo_2 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_M_2021.csv'
caminho_arquivo_3 = r'.\\separados por tipologia\\IntermediarioAdjacente\\IntermediarioAdjacente_ES_M_2022.csv'

df_1 = ler_processar_arquivo_csv(caminho_arquivo_1)
df_2 = ler_processar_arquivo_csv(caminho_arquivo_2)
df_3 = ler_processar_arquivo_csv(caminho_arquivo_3)

In [17]:
# Adicionar colunas padrão aos DataFrames
df_1 = adicionar_colunas_ausentes(df_1)
df_2 = adicionar_colunas_ausentes(df_2)
df_3 = adicionar_colunas_ausentes(df_3)

In [18]:
# Juntando os df e divdindo por 3 
df_resultado = processar_dataframes(df_1, df_2, df_3)
IA_M_2021 = df_resultado

C:\Users\Administrador\AppData\Local\Temp\ipykernel_12768\4064576934.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
  df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')


In [19]:
IA_M_2021.head(1)

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.0,0.333333,0.0,0.0,0.0,0.333333,0.333333,0.333333,1.666667,0.666667,1.0,0.666667,3.333333,5.666667,4.666667,3.333333,4.0,10.666667,37.0


-------

# Intermediario Remoto NÃO TEM MUNICIPIOS PARA A TIPOLOGIA

-------

# Rural Adjacente


## Feminino

* **Resposta para 2018**

In [20]:
# lendo arquivos 
caminho_arquivo_1 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_F_2017.csv'
caminho_arquivo_2 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_F_2018.csv'
caminho_arquivo_3 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_F_2019.csv'

df_1 = ler_processar_arquivo_csv(caminho_arquivo_1)
df_2 = ler_processar_arquivo_csv(caminho_arquivo_2)
df_3 = ler_processar_arquivo_csv(caminho_arquivo_3)

In [21]:
# Adicionar colunas padrão aos DataFrames
df_1 = adicionar_colunas_ausentes(df_1)
df_2 = adicionar_colunas_ausentes(df_2)
df_3 = adicionar_colunas_ausentes(df_3)

In [22]:
# Juntando os df e divdindo por 3 
df_resultado = processar_dataframes(df_1, df_2, df_3)
RA_F_2018 = df_resultado

C:\Users\Administrador\AppData\Local\Temp\ipykernel_12768\4064576934.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
  df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')


In [23]:
RA_F_2018.head(1)

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.666667,1.0,0.0,1.0,1.333333,5.0


* **Resposta para 2021**

In [24]:
# lendo arquivos 
caminho_arquivo_1 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_F_2020.csv'
caminho_arquivo_2 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_F_2021.csv'
caminho_arquivo_3 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_F_2022.csv'

df_1 = ler_processar_arquivo_csv(caminho_arquivo_1)
df_2 = ler_processar_arquivo_csv(caminho_arquivo_2)
df_3 = ler_processar_arquivo_csv(caminho_arquivo_3)

In [25]:
# Adicionar colunas padrão aos DataFrames
df_1 = adicionar_colunas_ausentes(df_1)
df_2 = adicionar_colunas_ausentes(df_2)
df_3 = adicionar_colunas_ausentes(df_3)

In [26]:
# Juntando os df e divdindo por 3 
df_resultado = processar_dataframes(df_1, df_2, df_3)
RA_F_2021 = df_resultado

C:\Users\Administrador\AppData\Local\Temp\ipykernel_12768\4064576934.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
  df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')


In [27]:
RA_F_2021.head(1)

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.333333,1.0,0.666667,0.333333,2.333333,2.333333,2.666667,4.333333,5.0,11.333333,30.666667


## Masculino

* **Resposta para 2018**

In [28]:
# lendo arquivos 
caminho_arquivo_1 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_M_2017.csv'
caminho_arquivo_2 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_M_2018.csv'
caminho_arquivo_3 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_M_2019.csv'

df_1 = ler_processar_arquivo_csv(caminho_arquivo_1)
df_2 = ler_processar_arquivo_csv(caminho_arquivo_2)
df_3 = ler_processar_arquivo_csv(caminho_arquivo_3)

In [29]:
# Adicionar colunas padrão aos DataFrames
df_1 = adicionar_colunas_ausentes(df_1)
df_2 = adicionar_colunas_ausentes(df_2)
df_3 = adicionar_colunas_ausentes(df_3)

In [30]:
# Juntando os df e divdindo por 3 
df_resultado = processar_dataframes(df_1, df_2, df_3)
RA_M_2018 = df_resultado

C:\Users\Administrador\AppData\Local\Temp\ipykernel_12768\4064576934.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
  df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')


In [31]:
RA_M_2018.head(1)

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.0,0.666667,0.666667,0.666667,1.666667,1.0,0.333333,0.333333,0.666667,1.666667,8.333333


* **Resposta para 2021**

In [32]:
# lendo arquivos 
caminho_arquivo_1 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_M_2020.csv'
caminho_arquivo_2 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_M_2021.csv'
caminho_arquivo_3 = r'.\\separados por tipologia\\RuralAdjacente\\RuralAdjacente_ES_M_2022.csv'

df_1 = ler_processar_arquivo_csv(caminho_arquivo_1)
df_2 = ler_processar_arquivo_csv(caminho_arquivo_2)
df_3 = ler_processar_arquivo_csv(caminho_arquivo_3)

In [33]:
# Adicionar colunas padrão aos DataFrames
df_1 = adicionar_colunas_ausentes(df_1)
df_2 = adicionar_colunas_ausentes(df_2)
df_3 = adicionar_colunas_ausentes(df_3)

In [34]:
# Juntando os df e divdindo por 3 
df_resultado = processar_dataframes(df_1, df_2, df_3)
RA_M_2021 = df_resultado

C:\Users\Administrador\AppData\Local\Temp\ipykernel_12768\4064576934.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
  df_resultado_soma['Causa - CID-BR-10'] = df_resultado_soma['Causa - CID-BR-10'].str.replace(r'^\.+', '')


In [35]:
RA_M_2021.head(1)

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.666667,1.0,1.0,2.666667,1.0,2.666667,5.333333,6.0,6.0,11.333333,38.0


-------

# Rural Remoto NÃO TEM MUNICIPIOS PARA A TIPOLOGIA


-------

# Urbano NÃO TEM MUNICIPIOS PARA A TIPOLOGIA


-------

# Parte final

Armazenando cada df em uma aba do arquivo 

In [36]:
# Criar um escritor Excel
with pd.ExcelWriter(r'.\\Triênio\\ES_trienio.xlsx', engine='xlsxwriter') as writer:
    # intermediario adjecente 
    IA_F_2018.to_excel(writer, sheet_name='IA_F_2018', index=False)
    IA_F_2021.to_excel(writer, sheet_name='IA_F_2021', index=False)
    IA_M_2018.to_excel(writer, sheet_name='IA_M_2018', index=False)
    IA_M_2021.to_excel(writer, sheet_name='IA_M_2021', index=False)
    # intermediario Remoto
    #IR_F_2018.to_excel(writer, sheet_name='IR_F_2018', index=False)
    #IR_F_2021.to_excel(writer, sheet_name='IR_F_2021', index=False)
    #IR_M_2018.to_excel(writer, sheet_name='IR_M_2018', index=False)
    #IR_M_2021.to_excel(writer, sheet_name='IR_M_2021', index=False)
    # Rural Adjacente 
    RA_F_2018.to_excel(writer, sheet_name='RA_F_2018', index=False)
    RA_F_2021.to_excel(writer, sheet_name='RA_F_2021', index=False)
    RA_M_2018.to_excel(writer, sheet_name='RA_M_2018', index=False)
    RA_M_2021.to_excel(writer, sheet_name='RA_M_2021', index=False)
    # Rural Remoto 
    #RR_F_2018.to_excel(writer, sheet_name='RR_F_2018', index=False)
    #RR_F_2021.to_excel(writer, sheet_name='RR_F_2021', index=False)
    #RR_M_2018.to_excel(writer, sheet_name='RR_M_2018', index=False)
    #RR_M_2021.to_excel(writer, sheet_name='RR_M_2021', index=False)
    # Urbano
    #U_F_2018.to_excel(writer, sheet_name='U_F_2018', index=False)
    #U_F_2021.to_excel(writer, sheet_name='U_F_2021', index=False)
    #U_M_2018.to_excel(writer, sheet_name='U_M_2018', index=False)
    #U_M_2021.to_excel(writer, sheet_name='U_M_2021', index=False)